#### This notebook contains code used to populate data dictionary
**Caution Note:** whenever an excel write function is executed the file is overwritten!

It is done step by step with different chunks of calls.

It is in development state and not cleaned at all!

In [9]:
import pandas as pd
from pandas_datareader import wb
from utils import append_df_to_excel
from data_in.legacy_data import prepare_mapping
import pandasdmx as sdmx

In [2]:
# read one sheet from excel tmee database provided by Eduard (and modified by Beto)
path_read_file = '../../data_ingestion/TM Indicators Flow Revised Database - 2020306 SP ED ECD -working file v-6 Aug 2020 (2).xlsx'
path_read_file_beto = './populate_data_dictionary/TM Indicators Flow Revised Database - 2020306 SP ED ECD -working file v-6 Aug 2020_beto.xlsx'
indicators = pd.read_excel(path_read_file, sheet_name='TM Revised database', header = 2)

#### Indicators New or Retained

In [3]:
# filter indicators that are new or retained from excel tmee data base
logic_not_removed = indicators['Status in the new `db'].str.lower().str.contains('new|retained')
# filter logic_not_removed that are not boolean
logic_not_null = logic_not_removed.notnull()
# operator AND for two logics above
logic_not_removed_null = logic_not_removed & logic_not_null
not_removed_indicators = indicators[logic_not_removed_null]

#### Indicators specified with Unesco Institute for Statistics (UIS) API source address

In [ ]:
# point to indicators that are extracted from UIS
logic_UIS = not_removed_indicators['Data Source'].str.lower().str.contains("api.uis.unesco.org")
# filter not boolean from logic_UIS
logic_UIS_not_null = logic_UIS.notnull()
# operator AND for two logics above
logic_UIS_not_null = logic_UIS & logic_UIS_not_null
print(f"There are {logic_UIS_not_null.sum()} indicators extracted from UIS.")

In [ ]:
# create new column in dataframe with their corresponding codes
not_removed_indicators['Code'] = not_removed_indicators['Indicator Code'][logic_UIS_not_null].str.replace(' ','_')

#### Indicators specified with API source in Helix

In [ ]:
# point to indicators with helix: or helix code: in excel tmee data base
logic_helix = not_removed_indicators['Data Source'].str.lower().str.contains("helix")
# filter logic_helix that are not boolean
logic_helix_not_null = logic_helix.notnull()
# operator AND for two logics above
logic_helix_not_null = logic_helix & logic_helix_not_null
print(f"There are {logic_helix_not_null.sum()} indicators extracted from Helix.")

In [ ]:
# populate dataframe with the corresponding codes from Helix Source
# before: eliminate all Helix ... prefix
not_removed_indicators['Code'][logic_helix_not_null] = not_removed_indicators[logic_helix_not_null]['Data Source']\
.str.replace(r'^.*?: ', '', regex=True)\
.str.replace('/', '').str.replace('\\', '')
# consider something more elegant with regex for the line above?
# consider something stronger with respect to blank spaces in the regex line!

#### Write Source names for Helix & UIS

In [ ]:
# create new column in dataframe for the corresponding Source Names
# standardize helix Api source names adding 'Helix:' prefix to code names
not_removed_indicators['Source Name'] = 'Helix: ' + not_removed_indicators[logic_helix_not_null]['Code']

In [ ]:
# standardize UIS Api source names adding 'UIS:' prefix to code names
not_removed_indicators['Source Name'][logic_UIS_not_null] = 'UIS: ' + not_removed_indicators[logic_UIS_not_null]['Code']

#### Get Indicators Addresses, retrieved by UIS API

In [ ]:
sources_uis = not_removed_indicators['Data Source'][logic_UIS_not_null]
# Keep from sources only the address 'root' before country specification
sources_uis = sources_uis.str.replace(r'^.*?https', 'https', regex=True).apply(lambda x: x.split('AL+')[0])
# note: the logic above doesn't work when indicator data source cell contains a new line before the address

#### Get Indicators retrieved by NSI

In [ ]:
# point to indicators that contains NSI in Data Source column (excel tmee data base)
logic_nsi = not_removed_indicators['Data Source'].str.lower().str.contains("nsi")
# filter logic_nsi that are not boolean
logic_nsi_not_null = logic_nsi.notnull()
# operator AND for two logics above
logic_nsi_not_null = logic_nsi & logic_nsi_not_null
print(f"There are {logic_nsi_not_null.sum()} indicators extracted from NSI.")

#### Match *new* or *retained* indicators processed as legacy

In [ ]:
path_content = './data_in/legacy_data/content_legacy_codes.csv'
nsi_new_or_retained = not_removed_indicators.Indicator[logic_nsi_not_null].values
tol_fuzzy = 0.85
nsi_match_legacy = prepare_mapping.match_legacy_and_new(path_content,nsi_new_or_retained,tol_fuzzy)

In [ ]:
tol_fuzzy = 0.85
n_match = (nsi_match_legacy['legacy match'] == "").sum()
print(f"There are {n_match} indicators matched for legacy and NSI retained/new at {tol_fuzzy*100}% fuzzy tolerance.")
print(f"This represents {round(n_match/logic_nsi_not_null.sum()*100,2)}% of the total NSI retained/new.")

In [ ]:
path_write = './populate_data_dictionary/'
file_write = 'legacy_nsi_match_direct'
nsi_match_legacy.to_csv(f"{path_write}{file_write}.csv",index=False)

##### Add column with status
To identify if indicators are labelled as `retained` or `new`.

Append these labels to existing Excel - NOTE: append function eliminate Excel comments =( -

In [ ]:
label_new_or_retained_nsi = not_removed_indicators['Status in the new `db'][logic_nsi_not_null]
# path to excel file for appending
path_append_file = './populate_data_dictionary/legacy_nsi_match.xlsx'
# use the function to append data dictionary
append_df_to_excel(path_append_file, label_new_or_retained_nsi, sheet_name='legacy_nsi_match', startrow=1,
                   startcol = 2, header = False, index = False)

In [ ]:
# Now read the processed file in Excel to comprobate changes in matched indicators
path_read = './populate_data_dictionary/'
file_read = 'legacy_nsi_match'
nsi_match_legacy_df = pd.read_csv(f"{path_read}{file_read}.csv", dtype=str)

In [ ]:
n_match_revised = nsi_match_legacy_df['legacy match'].notna().sum()
print(f"There are {n_match_revised} indicators matched for legacy and NSI retained.")
print(f"This represents {round(n_match_revised/(logic_nsi_not_null.sum()-2)*100,2)}% of the total NSI retained.")


##### NSI source with `new` in status
These are only two and comes from strings 'dimeNSIon' and 'coNSIst', not really 'NSI'

In [ ]:
label_new_or_retained_nsi.str.lower().str.contains('new').sum()
# 'nsi' in not_removed_indicators.loc[174,'Data Source'].lower()
# logic_nsi_not_null[174]
# label_new_or_retained_nsi.head(20)

In [ ]:
content_df = pd.read_csv(path_content, dtype=str)
content_df.indicator.replace(to_replace=r"\d+\.\d+.\d+.", value='', regex=True)

#### Search `WB` indicators

In [76]:
# point to indicators that contains world bank in Data Source column (excel tmee data base)
logic_wb = not_removed_indicators['Data Source'].str.lower().str.contains("wb|data reader|worldbank")
# filter logic_wb that are not boolean
logic_wb_not_null = logic_wb.notnull()
# operator AND for two logics above
logic_wb_not_null = logic_wb & logic_wb_not_null
print(f"There are {logic_wb_not_null.sum()} indicators extracted from world bank.")

There are 55 indicators extracted from world bank.


##### Indicators from world bank included in data dictionary
Extraction: pandas data reader

In [78]:
logic_dr = not_removed_indicators['Data Source'].str.lower().str.contains("worldbank")
# filter logic_dr that are not boolean
logic_dr_not_null = logic_dr.notnull()
# operator AND for two logics above
logic_dr_not_null = logic_dr & logic_dr_not_null
print(f"There are {logic_dr_not_null.sum()} indicators extracted from data reader.")

There are 18 indicators extracted from data reader.


5 of them are sex dissagregated, which become 15 in turn.

#### Search `ILO` indicators

In [6]:
# point to indicators that contains ILO in Data Source column (excel tmee data base)
logic_ilo = not_removed_indicators['Data Source'].str.lower().str.contains("ilo")
# filter logic_ilo that are not boolean
logic_ilo_not_null = logic_ilo.notnull()
# operator AND for two logics above
logic_ilo_not_null = logic_ilo & logic_ilo_not_null
print(f"There are {logic_ilo_not_null.sum()} indicators extracted from ILO.")

There are 41 indicators extracted from ILO.


#### Search `EUROSTAT` indicators (excluding `Eurostat & WB`)

In [7]:
# point to indicators that contains EUROSTAT in Data Source column (excel tmee data base)
logic_es = not_removed_indicators['Data Source'].str.lower().str.contains("eurostat")
# filter logic_es that are not boolean
logic_es_not_null = logic_es.notnull()
# operator AND for two logics above
logic_es_not_null = logic_es & logic_es_not_null
# number of 'Eurostat & WB' sources
n_es_and_wb = 26
print(f"There are {logic_es_not_null.sum() - n_es_and_wb} indicators extracted from EUROSTAT (excluding WB).")

There are 37 indicators extracted from EUROSTAT (excluding WB).


#### Search `WHO` indicators

In [8]:
# point to indicators that contains who (excel tmee data base)
logic_who = not_removed_indicators['Data Source'].str.lower().str.contains("who")
# filter logic_es that are not boolean
logic_who_not_null = logic_who.notnull()
# operator AND for two logics above
logic_who_not_null = logic_who & logic_who_not_null
# number of sources which the word 'who' appears
n_word_who = 1
print(f"There are {logic_who_not_null.sum() - n_word_who} indicators extracted from WHO.")

There are 5 indicators extracted from WHO.


#### Prepare Excel file to fill with information required
Silvia has kindly joined to complete this task.

In [71]:
# join all indicator names list (sources: WB, ILO, EUROSTAT, WHO)
third_party_indicators = pd.concat([not_removed_indicators['Indicator'][logic_wb_not_null],
                                  not_removed_indicators['Indicator'][logic_ilo_not_null],
                                  not_removed_indicators['Indicator'][logic_es_not_null],
                                  not_removed_indicators['Indicator'][logic_who_not_null]])
third_party_sources = pd.concat([not_removed_indicators['Data Source'][logic_wb_not_null],
                                  not_removed_indicators['Data Source'][logic_ilo_not_null],
                                  not_removed_indicators['Data Source'][logic_es_not_null],
                                  not_removed_indicators['Data Source'][logic_who_not_null]])
third_party_df = pd.Series(third_party_indicators.values, name='indicators').to_frame().join(pd.Series(third_party_sources.values, name='sources'))

In [72]:
path_write_file = './populate_data_dictionary/TMEE_third_party_sources.xlsx'
# use a function from stackoverflow to append data dictionary
append_df_to_excel(path_write_file, third_party_df, sheet_name='Indicators', header = False, index = False)

In [30]:
len(third_party_indicators)

165

In [92]:
65+41+37

143

In [ ]:
print(not_removed_indicators['Indicator'][logic_dr_not_null].values)

In [83]:
(wb.search(string='poverty', field='name', case=False).id.values == 'SI.POV.UMIC.NO').sum()

0

In [ ]:
dataWB = wb.download(indicator=['SI.POV.GINI','per_si_allsi.ben_q1_tot','per_si_allsi.cov_q1_tot','per_sa_allsa.cov_q1_tot','SM.POP.NETM','EN.ATM.PM25.MC.M3'], country=['JPN', 'MEX', 'RUS', 'TUR', 'ARG', 'ARM',  'BIH', 'BGR', 'EST', 'GEO', 'KAZ', 'KGZ', 'LTU', 'MNE', 'POL', 'SVK', 'SVN', 'TJK', 'TKM', 'UKR', 'UZB'], start=2010, end=2020)

In [90]:
wb.download(indicator='SI.POV.UMIC.NO', country=['KGZ'], start=1950, end=2050).head(100)

SI.POV.UMIC.NO
country         year                
Kyrgyz Republic 2019             NaN
                2018             3.5
                2017             3.7
                2016             3.7
                2015             3.8
                2014             3.5
                2013             3.7
                2012             3.6
                2011             3.3
                2010             3.3
                2009             3.1
                2008             3.1
                2007             3.5
                2006             3.5
                2005             4.1
                2004             3.8
                2003             4.6
                2002             4.6
                2001             4.6
                2000             4.5
                1999             NaN
                1998             3.6
                1997             NaN
                1996             NaN
                1995             NaN
                1994             NaN
                1993             NaN
                1992             NaN
                1991             NaN
                1990             NaN
                1989             NaN
                1988             NaN
                1987             NaN
                1986             NaN
                1985             NaN
                1984             NaN
                1983             NaN
                1982             NaN
                1981             NaN
                1980             NaN
                1979             NaN
                1978             NaN
                1977             NaN
                1976             NaN
                1975             NaN
                1974             NaN

In [ ]:
dataWB.loc[('Argentina', '2010')]

In [ ]:
wb_sdmx = sdmx.Request('WB')

In [ ]:
flow_msg = wb_sdmx.dataflow()
flow_msg.dataflow.DF_WITS_Tariff_TRAINS

In [ ]:
dataflows = sdmx.to_pandas(flow_msg.dataflow)
dataflows

In [ ]:
(wb.get_indicators().id == '3.0.Gini').sum()

#### Research
##### Adding more possible Helix!
There are three marked as 'Helix' only, plus: `ECD_CHLD_U5_BKS-HM`, `ECD_CHLD_U5_PLYTH-HM` and 'SOWC 2019'
* Marked as 'Helix' only worked fine, end up being Unicef WS indicators (`WS_SCH_H-B`, `WS_SCH_S-B` and `WS_SCH_W-B`).
* Indicators `ECD_CHLD_U5_BKS-HM` and `ECD_CHLD_U5_PLYTH-HM` also work fine!
* SOWC 2019 haven't found (net migration rate 2015-2020, supposed to be calculated by UNICEF)

In [ ]:
logic_more_helix = not_removed_indicators['Data Source'].str.lower().str.contains("sowc 2019")
logic_more_helix = logic_more_helix & logic_more_helix.notnull()
not_removed_indicators[logic_more_helix]

##### World Bank
Those that are already there, I will add them with pd data reader and the code.

In [ ]:
indicator_name = 'LO.PISA.MAT'
pisa_mat = wb.download(indicator=indicator_name, country=['AL'], start=1950, end=2020)
indicator_name = 'LO.PISA.MAT.FE'
pisa_mat_fe = wb.download(indicator=indicator_name, country=['AL'], start=1970, end=2020)
indicator_name = 'LO.PISA.MAT.MA'
pisa_mat_ma = wb.download(indicator=indicator_name, country=['AL'], start=1970, end=2020)

In [ ]:
indicator_name = 'LO.PISA.MAT'
pisa_mat = wb.download(indicator=indicator_name, start=1970, end=2070)
indicator_name = 'LO.PISA.MAT.FE'
pisa_mat_fe = wb.download(indicator=indicator_name, start=1970, end=2070)
indicator_name = 'LO.PISA.MAT.MA'
pisa_mat_ma = wb.download(indicator=indicator_name, start=1970, end=2070)

In [ ]:
pd_all = pd.concat([pisa_mat,pisa_mat_fe,pisa_mat_ma])
pd_all

#### Working out what remains to be Extracted!
So far we have API's UIS and Helix

In [ ]:
logic_not_helix_uis = ~(logic_helix_not_null | logic_UIS_not_null)

In [ ]:
# this is basically the info to process
not_removed_indicators['Data Source'][logic_not_helix_uis].unique()

In [ ]:
# indicators with no info
not_removed_indicators['Data Source'][logic_not_helix_uis].isna().sum()

In [ ]:
# indicators extracted from NSI
not_removed_indicators['Data Source'][logic_not_helix_uis].str.lower().str.contains("nsi").sum()

In [ ]:
# indicators extracted from calculations
not_removed_indicators['Data Source'][logic_not_helix_uis].str.lower().str.contains("calculated").sum()

In [ ]:
# indicators from 'NSI or/and EuroStat'
print(not_removed_indicators['Data Source'][logic_not_helix_uis].str.lower().str.contains(" eurostat").sum())
# indicators from 'Eurostat & WB'
print(not_removed_indicators['Data Source'][logic_not_helix_uis].str.lower().str.contains("eurostat ").sum())
# indicators from 'Eurostat'
print(not_removed_indicators['Data Source'][logic_not_helix_uis].str.lower().str.contains("eurostat").sum())

In [ ]:
# indicators extracted from WB or 'Eurostat & WB'
print(not_removed_indicators['Data Source'][logic_not_helix_uis].str.lower().str.contains("wb").sum())
# indicators with address containing 'worldbank'
print(not_removed_indicators['Data Source'][logic_not_helix_uis].str.lower().str.contains("worldbank").sum())
# indicators marked as data reader comes from world bank
print(not_removed_indicators['Data Source'][logic_not_helix_uis].str.lower().str.contains("data reader").sum())

In [ ]:
# contains just 'WB' or 'Eurostat & WB' and nothing else (no source specified)
print((not_removed_indicators['Data Source'][logic_not_helix_uis] == 'WB').sum())
print((not_removed_indicators['Data Source'][logic_not_helix_uis] == 'Eurostat & WB').sum())

In [ ]:
# contains api in the address (these are all world bank API)
print(not_removed_indicators['Data Source'][logic_not_helix_uis].str.lower().str.contains("api").sum())
print(f"From WB only {round(11/57*100,2)}% specifies API")

In [ ]:
# indicators extracted from WHO
not_removed_indicators['Data Source'][logic_not_helix_uis].str.lower().str.contains("who").sum()

In [ ]:
# indicators extracted from ILO
not_removed_indicators['Data Source'][logic_not_helix_uis].str.lower().str.contains("ilo").sum()

In [ ]:
# contains just 'ILO' and nothing else (no source specified)
print((not_removed_indicators['Data Source'][logic_not_helix_uis] == 'ILO').sum())
print(f"From ILO only {round(1/41*100,2)}% specifies API")

In [ ]:
# indicators to search in Helix
not_removed_indicators['Data Source'][logic_not_helix_uis].str.lower().str.contains("helix").sum()

In [ ]:
# indicators to search in UIS
not_removed_indicators['Data Source'][logic_not_helix_uis].str.lower().str.contains("uis").sum()

In [ ]:
# indicators marked with MICS
not_removed_indicators['Data Source'][logic_not_helix_uis].str.lower().str.contains("mics").sum()

In [ ]:
# indicators marked with SMQs
not_removed_indicators['Data Source'][logic_not_helix_uis].str.lower().str.contains("smq").sum()

In [ ]:
# indicators to search in TM
print(not_removed_indicators['Data Source'][logic_not_helix_uis].str.lower().str.contains("tm").sum())
# indicators from web not scrapable
print(not_removed_indicators['Data Source'][logic_not_helix_uis].str.lower().str.contains("not scrap").sum())
# indicators from web not scrapable
print(not_removed_indicators['Data Source'][logic_not_helix_uis].str.lower().str.contains("ptb").sum())

#### Appends to Data Dictionary
**Caution Note:** whenever an excel write function is executed the file is overwritten!
##### Append info Indicators Code - API's from UIS & Helix

In [ ]:
# path to file for appending
path_write_file = './populate_data_dictionary/indicator_dictionary_TM_v2.xlsx'

In [ ]:
# use a function from stackoverflow to append data dictionary
append_df_to_excel(path_write_file, not_removed_indicators.iloc[21:,[105]], sheet_name='Indicator', startrow=22,
                   startcol = 5, header = False, index = False)

##### Append info Indicators Source Name - API's from UIS & Helix

In [ ]:
# place list of helix and UIS api's in source
logic_helix_uis = not_removed_indicators['Source Name'].notna()
append_df_to_excel(path_write_file, not_removed_indicators.loc[logic_helix_uis,'Source Name'].iloc[2:],
                   sheet_name='Source', startrow=6,
                   startcol = 2, header = False, index = False)

##### Append info Indicators Addresses - UIS API

In [ ]:
# place list UIS api's addresses in source spreadsheet
append_df_to_excel(path_write_file, sources_uis,
                   sheet_name='Source', startrow=26,
                   startcol = 3, header = False, index = False)

#### Dev testing zone

In [ ]:
from sdmx import sdmx_struc
from fileUtils import fileDownload
import numpy as np

In [ ]:
logic_wb = not_removed_indicators['Data Source'].str.lower().str.contains("worldbank")
# filter logic_dr that are not boolean
logic_wb_not_null = logic_wb.notnull()
# operator AND for two logics above
logic_wb_not_null = logic_wb & logic_wb_not_null
print(np.r_[not_removed_indicators['Indicator'][logic_wb_not_null].values[0:1],
      not_removed_indicators['Indicator'][logic_wb_not_null].values[2:]])

In [ ]:
url_endpoint = 'https://sdmx.data.unicef.org/ws/public/sdmxapi/rest/data/ECARO,TRANSMONEE,1.0/'
api_address = url_endpoint + 'all'
api_params = {'format':'sdmx-json', 'detail':'structureOnly'}
# API dataflow structure request
d_flow_struc = fileDownload.api_request(api_address,api_params)

In [ ]:
country_codes_3 = sdmx_struc.get_all_country_codes(d_flow_struc.json())
country_codes_3.values()

In [ ]:
sources_uis

In [ ]:
not_removed_indicators.loc[logic_helix_uis,'Source Name'].iloc[2:]

In [ ]:
not_removed_indicators.iloc[105]

In [ ]:
# create new column in dataframe with codes with add
# get indicator names

In [ ]:
# Keep from sources only the address 'root' before country specification
sources_uis[logic_api].str.replace(r'^.*?https', 'https', regex=True).apply(lambda x: x.split('AL+')[0])
# 'https://api.uis.unesco.org/sdmx/data/UNESCO,SDG4,2.0/PQTR..L3................'
# 'https://api.uis.unesco.org/sdmx/data/UNESCO,SDG4,2.0/PQTR..L3................'

In [ ]:
sources_uis = not_removed_indicators['Data Source'][logic_UIS_not_null]
logic_api = sources_uis.str.lower().str.contains("api.uis.unesco.org")

In [ ]:
# not_removed_indicators['Code'] = not_removed_indicators[logic_UIS_not_null]['Data Source']